<a href="https://colab.research.google.com/github/matiaszabal/argand/blob/master/simula_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat Bot Evaluation as Multi-agent Simulation using Llama 3 with Ollama and LangChain

When building a chat bot, such as a customer support assistant, it can be hard to properly evaluate your bot's performance. It's time-consuming to have to manually interact with it intensively for each code change.

One way to make the evaluation process easier and more reproducible is to simulate a user interaction.

With LangChain, it's easy to set this up. Below is an example of how to create a "virtual user" to simulate a conversation using the Llama 3 model locally via Ollama.

---

## How to Use This Notebook

1. **Install Ollama and LangChain**: Ensure you have the Ollama application installed and configured to serve the Llama 3 model locally. Also, install the `langchain` library.

   ```bash
   pip install langchain ollama
   ```

2. **Set Up Ollama**: Make sure the Llama 3 model is correctly installed in Ollama and running.

3. **Adjust Model Name if Necessary**: If your Llama 3 model has a different name in Ollama, update the model parameter when initializing `Ollama`.

   ```python
   llm = Ollama(model="your_model_name")
   ```

4. **Run the Notebook**: Execute each cell in order to simulate the conversation between the chat bot and the simulated user.

5. **Customize Instructions**: You can modify the `instructions` variable to change the simulated user's behavior.

---


In [14]:
# Install necessary packages
#!pip install -U langchain ollama
!curl -sSfL https://ollama.ai/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
!pip install langchain ollama langchain_community
import os
from typing import List, Dict, Any

from langchain import LLMChain, PromptTemplate
from langchain.llms import Ollama
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00


In [10]:
# Initialize the Llama 3 model via Ollama
llm = Ollama(model="llama3")  # Replace 'llama3' with your model name if different

<ipython-input-10-3a37dee465ea>:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")  # Replace 'llama3' with your model name if different


In [11]:
def my_chat_bot(messages: List[Dict[str, Any]]) -> Dict[str, Any]:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    # Prepare the conversation history
    messages = [system_message] + messages
    # Convert messages to prompt
    prompt = ""
    for msg in messages:
        prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"
    prompt += "Assistant:"

    # Get response from the Llama 3 model via Ollama
    response = llm(prompt).strip()

    return {"role": "assistant", "content": response}

In [19]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...  38% ▕▏ 1.8 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  39% ▕▏ 1.8 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  39% ▕▏ 1.8 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  39% ▕▏ 1.8 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  40% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  40% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  41% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  41% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  41% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  41% ▕▏ 1.9 GB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  41% ▕▏ 1.9 GB/4.7 GB  154 MB/s   

In [26]:
# Test the chat bot with a simple message
response = my_chat_bot([{"role": "user", "content": "Hi!"}])
print(response)

{'role': 'assistant', 'content': "Hi there! Welcome to [Airline Name] Customer Support. My name is Emma, and I'll be happy to help you with any questions or concerns you may have. How can I assist you today? Are you traveling with us soon or do you need some general information about our airline or services?"}


In [45]:
# Define the system prompt template for the simulated user
system_prompt_template = '''You are a customer of an airline company.
You are interacting with a user who is a customer support person.

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'.'''

# Import necessary classes from langchain.schema.messages
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage  # Import necessary classes
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate
)

# Create the prompt template
# Include a placeholder for the user's input ("input") along with "history"
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_prompt_template),
        MessagesPlaceholder(variable_name="history"),
        # Add a placeholder for the user's input
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

# Instructions for the simulated user
instructions = '''Your name is Harrison. You are trying to get a refund for the trip you took to Alaska.
You want them to give you ALL the money back.
This trip happened 5 years ago.'''

# Partial the prompt with instructions
prompt = prompt.partial(instructions=instructions)

# Initialize the conversation chain with memory
from langchain.memory import ConversationBufferMemory # Import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
from langchain.chains import ConversationChain # Import ConversationChain
simulated_user_chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False,
)

In [56]:
# Import necessary modules
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder  # Correct import
#from langchain.schema import SystemMessage, MessagesPlaceholder  # Correct import <---- Removed
from langchain.schema import SystemMessage  # Correct import
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Define the system prompt template for the simulated user
system_prompt_template = '''You are a customer of an airline company.
You are interacting with a user who is a customer support person.

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'.'''


# Create the prompt template
# Include a placeholder for the user's input ("input") along with "history"
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_prompt_template),
        MessagesPlaceholder(variable_name="history"),
        # Add a placeholder for the user's input
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

# Instructions for the simulated user
instructions = '''Your name is Harrison. You are trying to get a refund for the trip you took to Alaska.
You want them to give you ALL the money back.
This trip happened 5 years ago.'''

# Partial the prompt with instructions
prompt = prompt.partial(instructions=instructions)

# Initialize the conversation chain with memory
memory = ConversationBufferMemory(return_messages=True)
simulated_user_chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False,
)

In [60]:
# Import necessary modules
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder  # Correct import
#from langchain.schema import SystemMessage, MessagesPlaceholder  # Correct import <---- Removed
from langchain.schema import SystemMessage  # Correct import
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# ... (rest of your code) ...

In [61]:
# Import necessary modules
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,  # Correct import location
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# ... (rest of the code remains the same) ...

In [62]:
from langchain.prompts import HumanMessagePromptTemplate

In [66]:
# Define the system prompt template for the simulated user
system_prompt_template = '''You are a customer of an airline company.
You are interacting with a user who is a customer support person.

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'.'''

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate  # Import HumanMessagePromptTemplate
)

# Create the prompt template
# Include a placeholder for the user's input ("input") along with "history"
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_prompt_template),
        MessagesPlaceholder(variable_name="history"),
        # Add a placeholder for the user's input
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

# ... (rest of the code remains the same)

In [67]:
# Define the system prompt template for the simulated user
system_prompt_template = '''You are a customer of an airline company.
You are interacting with a user who is a customer support person.

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'.'''

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    # SystemMessage,  # Remove this incorrect import
    HumanMessagePromptTemplate  # Import HumanMessagePromptTemplate
)
from langchain.schema import SystemMessage # Import SystemMessage from the correct module

# Create the prompt template
# Include a placeholder for the user's input ("input") along with "history"
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_prompt_template),
        MessagesPlaceholder(variable_name="history"),
        # Add a placeholder for the user's input
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

# ... (rest of the code remains the same)

In [68]:
def chat_bot_node(conversation: List[BaseMessage]) -> AIMessage:
    # Convert conversation history to list of dicts
    messages = []
    for msg in conversation:
        if isinstance(msg, HumanMessage):
            messages.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            messages.append({"role": "assistant", "content": msg.content})
        elif isinstance(msg, SystemMessage):
            messages.append({"role": "system", "content": msg.content})

    # Get response from the chat bot
    response = my_chat_bot(messages)
    return AIMessage(content=response["content"])

def simulated_user_node(conversation: List[BaseMessage]) -> HumanMessage:
    # Swap roles in the conversation for the simulated user
    swapped_conversation = []
    for msg in conversation:
        if isinstance(msg, HumanMessage):
            swapped_conversation.append(AIMessage(content=msg.content))
        elif isinstance(msg, AIMessage):
            swapped_conversation.append(HumanMessage(content=msg.content))
        else:
            swapped_conversation.append(msg)

    # Update the memory
    memory.chat_memory.messages = swapped_conversation

    # Get response from the simulated user
    response = simulated_user_chain.predict(input="")
    return HumanMessage(content=response)

In [69]:
# Initialize conversation history
conversation = []

# Start the conversation with the chat bot's greeting
bot_greeting = chat_bot_node(conversation)
print(f"Chat Bot: {bot_greeting.content}")
conversation.append(bot_greeting)

# Conversation loop
while True:
    # Simulated user responds
    user_response = simulated_user_node(conversation)
    print(f"User: {user_response.content}")
    conversation.append(user_response)

    # Check for stopping condition
    if user_response.content.strip().upper() == "FINISHED":
        break

    # Chat bot responds
    bot_response = chat_bot_node(conversation)
    print(f"Chat Bot: {bot_response.content}")
    conversation.append(bot_response)

Chat Bot: Thank you for contacting our airline's customer support team! My name is Karen, and I'll be happy to assist you with any questions or concerns you may have. What can I help you with today? Are you traveling with us soon, or do you have a query about an existing booking?
User: Hi Karen, thank you for helping me! I'm having some issues with my upcoming flight from New York to Los Angeles on March 15th. My reservation number is #123456 and I was supposed to receive my boarding pass via email yesterday, but it never arrived. Can you please help me figure out what's going on?
Chat Bot: Thank you for reaching out! I apologize that your boarding pass didn't arrive as expected. Let me look into this further for you.

Can you confirm that you checked your spam or junk mail folder to ensure the email wasn't filtered out? Sometimes, emails can get misdirected into those folders.

Also, may I ask what type of device and browser are you using to access your account and check-in online? Th

## Conclusion

This notebook demonstrates how to simulate a chat bot evaluation using Llama 3 with Ollama and LangChain. By creating a simulated user, you can automate conversations to test and improve your chat bot's performance without manual intervention.

Feel free to customize the prompts, conversation logic, and stopping conditions to suit your specific use case.

---

**Note**: Ensure that the Llama 3 model is properly set up and accessible via Ollama. If you encounter any issues with the `Ollama` class or the `llm` object, confirm that your environment is correctly configured and that the model name matches the one you have installed.

If you have any questions or need further assistance, don't hesitate to ask!